In [1]:
import tensorflow as tf
import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
AUTOTUNE = tf.data.experimental.AUTOTUNE
import multiprocessing
import numpy as np
from IPython.display import clear_output
num_cpus = multiprocessing.cpu_count()
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU') 
tf.config.experimental.set_visible_devices(devices=my_devices, device_type='CPU')
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2023-10-01 12:31:37.461774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 12:31:38.310598: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-01 12:31:39.357639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:31:39.431764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4100)]
    )
logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
cpus = tf.config.list_physical_devices('CPU')
logical_cpus = tf.config.list_logical_devices('CPU')
print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPUs")

1 Physical GPU, 1 Logical GPUs
1 Physical CPU, 1 Logical CPUs


2023-10-01 12:31:39.440864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:31:39.441113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:31:39.441265: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
path_inp = "../data_for_gan/input"
path_out = "../data_for_gan/output"
input = []
output1 = []
output2 = []
for root, dirs, files in os.walk(path_inp):
    for file in files:
        input.append(os.path.join(root, file))
        output1.append(path_out+"/"+file[0]+"/"+file[0]+"1.png")
        output2.append(path_out+"/"+file[0]+"/"+file[0]+"2.png")

In [4]:
input, output1, output2

(['../data_for_gan/input/4.jpg',
  '../data_for_gan/input/3.jpg',
  '../data_for_gan/input/5.png',
  '../data_for_gan/input/1.jpg',
  '../data_for_gan/input/6.jpg'],
 ['../data_for_gan/output/4/41.png',
  '../data_for_gan/output/3/31.png',
  '../data_for_gan/output/5/51.png',
  '../data_for_gan/output/1/11.png',
  '../data_for_gan/output/6/61.png'],
 ['../data_for_gan/output/4/42.png',
  '../data_for_gan/output/3/32.png',
  '../data_for_gan/output/5/52.png',
  '../data_for_gan/output/1/12.png',
  '../data_for_gan/output/6/62.png'])

In [5]:
LR = 0.001
# Custom loss function
def bce_dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Привести y_true к типу float32
    # dice_loss = dice_coef(y_true, y_pred)
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce_loss

def dice_coef(y_true, y_pred, smooth=1e-5):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    return (2.0 * intersection + smooth) / (union + smooth)

NUM_EPOCHS=25
IMG_WIDTH       = 512
IMG_HEIGHT      = 512
IMG_CHANNELS    = 3

In [6]:
tf.keras.backend.clear_session()
nb_filter = np.array([32,64,128,256,512]) / 2
# Build U-Net++ model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)


c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.5) (c1)
c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
c1 = Dropout(0.5) (c1)
p1 = MaxPooling2D((2, 2), strides=(2, 2)) (c1)

c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.5) (c2)
c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
c2 = Dropout(0.5) (c2)
p2 = MaxPooling2D((2, 2), strides=(2, 2)) (c2)

up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(c2)
conv1_2 = concatenate([up1_2, c1], name='merge12', axis=3)
c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_2)
c3 = Dropout(0.5) (c3)
c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
c3 = Dropout(0.5) (c3)

conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
conv3_1 = Dropout(0.5) (conv3_1)
conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3_1)
conv3_1 = Dropout(0.5) (conv3_1)
pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
conv2_2 = concatenate([up2_2, c2], name='merge22', axis=3) #x10
conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_2)
conv2_2 = Dropout(0.5) (conv2_2)
conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_2)
conv2_2 = Dropout(0.5) (conv2_2)

up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
conv1_3 = concatenate([up1_3, c1, c3], name='merge13', axis=3)
conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_3)
conv1_3 = Dropout(0.5) (conv1_3)
conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_3)
conv1_3 = Dropout(0.5) (conv1_3)

conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pool3)
conv4_1 = Dropout(0.5) (conv4_1)
conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv4_1)
conv4_1 = Dropout(0.5) (conv4_1)
pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=3) #x20
conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3_2)
conv3_2 = Dropout(0.5) (conv3_2)
conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3_2)
conv3_2 = Dropout(0.5) (conv3_2)

up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
conv2_3 = concatenate([up2_3, c2, conv2_2], name='merge23', axis=3)
conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_3)
conv2_3 = Dropout(0.5) (conv2_3)
conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_3)
conv2_3 = Dropout(0.5) (conv2_3)

up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
conv1_4 = concatenate([up1_4, c1, c3, conv1_3], name='merge14', axis=3)
conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_4)
conv1_4 = Dropout(0.5) (conv1_4)
conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_4)
conv1_4 = Dropout(0.5) (conv1_4)

conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pool4)
conv5_1 = Dropout(0.5) (conv5_1)
conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv5_1)
conv5_1 = Dropout(0.5) (conv5_1)

up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=3) #x30
conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv4_2)
conv4_2 = Dropout(0.5) (conv4_2)
conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv4_2)
conv4_2 = Dropout(0.5) (conv4_2)

up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=3)
conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3_3)
conv3_3 = Dropout(0.5) (conv3_3)
conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3_3)
conv3_3 = Dropout(0.5) (conv3_3)

up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
conv2_4 = concatenate([up2_4, c2, conv2_2, conv2_3], name='merge24', axis=3)
conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_4)
conv2_4 = Dropout(0.5) (conv2_4)
conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_4)
conv2_4 = Dropout(0.5) (conv2_4)

up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
conv1_5 = concatenate([up1_5, c1, c3, conv1_3, conv1_4], name='merge15', axis=3)
conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_5)
conv1_5 = Dropout(0.5) (conv1_5)
conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1_5)
conv1_5 = Dropout(0.5) (conv1_5)

nestnet_output_4 = Conv2D(3, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal',  name='output_4', padding='same')(conv1_5)

model = Model([inputs], [nestnet_output_4])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5*1e-6), loss=bce_dice_loss)

2023-10-01 12:31:43.849028: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:753] failed to allocate 4.00GiB (4299161600 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [7]:
@tf.function
def load_image_and_mask(image_path, mask1_path, mask2_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH), method="bicubic")
    image = tf.cast(image, dtype=tf.float32) / 255.
    mask1 = tf.io.read_file(mask1_path)
    mask1 = tf.image.decode_png(mask1, channels=1)
    mask1 = tf.image.resize(mask1, (IMG_HEIGHT, IMG_WIDTH))
    mask1 = ~tf.cast(mask1, dtype=tf.uint8)
    mask2 = tf.io.read_file(mask2_path)
    mask2 = tf.image.decode_png(mask2, channels=1)
    mask2 = tf.image.resize(mask2, (IMG_HEIGHT, IMG_WIDTH))
    mask2 = ~tf.cast(mask2, dtype=tf.uint8)
    mask1 = tf.expand_dims(mask1, axis=0)  # Добавить размерность спереди
    mask2 = tf.expand_dims(mask2, axis=0)  # Добавить размерность спереди
    mask1 = tf.squeeze(mask1, axis=-1)
    mask2 = tf.squeeze(mask2, axis=-1)
    mask = tf.stack([(~mask1 & ~mask2), mask1, mask2], axis=-1)
    image = tf.expand_dims(image, axis=0)
    return image, mask

In [8]:
batch_size = 3
input_dataset = tf.data.Dataset.from_tensor_slices(input)
output1_dataset = tf.data.Dataset.from_tensor_slices(output1)
output2_dataset = tf.data.Dataset.from_tensor_slices(output2)

# Примените load_image_and_mask к каждому элементу внутри датасетов
# input_dataset = input_dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
# output1_dataset = output1_dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
# output2_dataset = output2_dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)

# Зипуйте датасеты вместе
dataset = tf.data.Dataset.zip((input_dataset, output1_dataset, output2_dataset))
dataset_train = dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [9]:
import matplotlib.pyplot as plt
for x,y in dataset_train:
    # for x,y,z in dat:
    # plt.imshow(x)
    # plt.show()
    # plt.imshow(y[:,:,:,0])
    # plt.show()
    # plt.imshow(y[:,:,:,1])
    # plt.show()
    # plt.imshow(y[:,:,:,2])
    # plt.show()
    print(x.shape, y.shape)
    # print(dat[0].shape, dat[1].shape, )

(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)


In [12]:
history = model.fit(dataset_train, epochs=1)

5/5 [==============================] - 3s 489ms/step - loss: 1.5385


In [13]:
model.save("mod", save_format='tf')

INFO:tensorflow:Assets written to: mod/assets


INFO:tensorflow:Assets written to: mod/assets
